In [1]:
import pandas as pd
import numpy as np
import json, os, uuid
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
MOUNT_DIR = '/content/drive/MyDrive/BarclaysReviews/'
DATA_DIR = '/content/drive/MyDrive/BarclaysReviews/data/'
MODEL_DIR = '/content/drive/MyDrive/BarclaysReviews/model/'


In [42]:
final_merged_reviews = pd.read_csv(DATA_DIR+"final_merged_reviews_cleaned.csv")

In [67]:
pd.set_option('display.max_colwidth', -1)
# final_merged_reviews[final_merged_reviews["cleaned_data"].notnull() & final_merged_reviews["cleaned_data"].str.contains("reward")][["review_description", "source","rating"]].rating.value_counts()
final_merged_reviews[final_merged_reviews["cleaned_data"].notnull() & final_merged_reviews["cleaned_data"].str.contains("blue")][["review_description", "source","rating"]].shape

(49, 3)

In [4]:
import spacy 
nlp = spacy.load('en_core_web_sm')
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
barclays_stop_list = ['app', 'barclays','need','will','pay','work']
stop_words.extend(barclays_stop_list)
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:

import re
wnl = WordNetLemmatizer()
def tokenizer(text):
    regex_num_ponctuation = '(\d+)|([^\w\s])'
    regex_little_words = r'(\b\w{1,2}\b)'
    return [wnl.lemmatize(token.lemma_.lower()) for token in nlp(text) if not token.is_stop and not token.is_punct ]

def LemmaTokenizer(doc):
      wnl = WordNetLemmatizer()
      regex_num_ponctuation = '(\d+)|([^\w\s])'
      regex_little_words = r'(\b\w{1,2}\b)'
      return [wnl.lemmatize(t) for t in word_tokenize(doc) 
              if not re.search(regex_num_ponctuation, t) and not re.search(regex_little_words, t) and not t.is_stop and not t.is_punct]

In [7]:
!pip install lda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.9/320.9 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 12.8 MB/s eta 0:00:00
  Created wheel for lda: filename=lda-2.0.0-cp310-cp310-linux_x86_64.whl size=628213 sha256=5c240b18a6bfb4144439249a73dd133a6cd75731384ee498a3ca92572e85090a
  Stored in directory: /root/.cache/pip/wheels/9d/32/fa/07fdf278a0b24269ce4d0be3d642c28dd0e888a4c5f2031279
Successfully built lda


In [8]:
!pip install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=c041364f57283628305c902883bb68108f137725b3aa201d3e51d0e5d7264696
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55640 sha256=5a63ad249cd0db690062714621393d4379594b1e43d5c6cfa1b5a457a05a275e
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent


In [9]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=1fd959e4b5d5b16badbb04144e4d3185552332d7b02b1ee57963eada302fdebd
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [10]:
!pip install HDBSCAN

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 59.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for HDBSCAN: filename=hdbscan-0.8.29-cp310-cp310-linux_x86_64.whl size=3541544 sha256=248cdeefe67c8e6e515f03ca4ac46bdbdce02a3218b285b96baeef65c1472946
  Stored in directory: /root/.cache/pip/wheels/dc/52/e3/6c6b60b126b4d5c4370cb5ac071b82950f91649d62d72f7f56
Successfully built HDBSCAN


In [14]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 12.5 MB/s eta 0:00:00


In [11]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=100, min_samples=40,
                        gen_min_span_tree=True,
                        prediction_data=True)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer 

In [15]:
from bertopic import BERTopic

In [16]:
import time

In [26]:
def topicModellingBERT(df,columns, textCol = "review_description",min_df=4,max_features = 10000,top_n_words=5,path="",model_name=""):
  print(columns)
  if bool(columns):
    # print("columns")
    for key in columns:
      # print(df.shape)
      df = df[df[key].str.contains(columns[key])]


  min_df = min_df # minimum required occurences of a word, e.g 4
  max_features = max_features # max number of unique words, e.g 10000

  # vectorizer_model = CountVectorizer(
  #               min_df=min_df, 
  #               max_features=max_features, 
  #               tokenizer=tokenizer, # use tokenizer we defined before
  #               ngram_range=(1,2),
  #               stop_words=stop_words) # we use unigram & bigram
  # X = vectorizer.fit_transform(final_merged_reviews_not_null['review_description'])


  df = df[df[textCol].notnull()]
      
  print('dataet length: ',df.shape,'\n')
  # # we add this to remove stopwords that can pollute topcs
  vectorizer_model = CountVectorizer(ngram_range=(1, 2), min_df=min_df,max_features=max_features)
  df.reset_index(drop=True, inplace=True)
  model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    top_n_words=5,
    language='english',
    calculate_probabilities=True,
    verbose=True
  )
  topics, probs = model.fit_transform(df[textCol])
  print(model.get_topic_info())
  
  model.save(path+model_name+'_'+str(int(time.time())))
  return  df,topics, probs, model



In [ ]:
#  topics, probs, model = topicModellingBERT(df=final_merged_reviews,sentiment_col="customer_sentiment",sentiment='NEGATIVE',path=MODEL_DIR,model_name='negative_topic_modelling')
 

In [ ]:
 df,topics, probs, model = topicModellingBERT(df=final_merged_reviews,textCol="cleaned_data",sentiment_col="customer_sentiment",sentiment='NEGATIVE',path=MODEL_DIR,model_name='negative_topic_modelling')
 

NEGATIVE Topic Modelling on :  customer_sentiment column
dataet length:  (6172, 20) 



Batches:   0%|          | 0/193 [00:00<?, ?it/s]

2023-04-30 08:50:56,465 - BERTopic - Transformed documents to Embeddings
2023-04-30 08:51:00,118 - BERTopic - Reduced dimensionality
2023-04-30 08:51:00,539 - BERTopic - Clustered reduced embeddings


    Topic  Count                                              Name
0      -1   2243                       -1_slow_account_bad_payment
1       0   1029  0_contactless_payment_contactless payment_device
2       1    471                           1_update_slow_phone_new
3       2    463                     2_cheque_photo_picture_camera
4       3    423                          3_chat_slow_service_help
5       4    268       4_calendar_calendar_view_notification_crash
6       5    241                         5_account_money_slow_year
7       6    229              6_account_register_open_open account
8       7    228                7_crash_login_constantly crash_bad
9       8    134                         8_slow_login_account_open
10      9    123                        9_navigate_fscs_open_crash
11     10    114             10_barclaycard_card_credicard_barclay
12     11    103               11_notification_money_payment_spend
13     12    103             12_transfer_money_transaction_acc

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
model.visualize_barchart(top_n_topics=12)
#rainbow login info
#word count 
# payment = topic_0,topic 10 -l1 mortgage - rewards
# crash = topic 1,4,7 -l1 calender (app or gplay)
# notifications = topic 11,4 - exclude calender
# cheque = topic 2
# login = topic 6, - error > error > customized ,  
# slow = topic 8,5 - 
#customer care -topic 3 chat new chat > crash

mortage = 49/6.5 k

In [ ]:
model.visualize_barchart(topics=[-1])

In [ ]:
model.visualize_barchart(topics=[-1])

In [ ]:
model.get_topic_info()

,Topic,Count,Name
0,-1,825,-1_update_account_open_new
1,0,1746,0_contactless_payment_payments_bank
2,1,1305,1_account_contactless_card_bank
3,2,415,2_cheque_cheques_photo_paying
4,3,275,3_slow_open_difficult_useless
5,4,240,4_chat_service_call_chat function
6,5,189,5_calendar_view_calendar view_calender
7,6,188,6_contactless_mobile_working_contactless mobile
8,7,173,7_crashes_crashing_keeps_keeps crashing
9,8,151,8_update_slow_updates_every


In [29]:
model.visualize_hierarchy()

In [ ]:
 import pickle

In [ ]:
with open(MODEL_DIR+'topic_negative_2.pkl', 'wb') as f:
  pickle.dump(topics, f)

In [ ]:
# with open('parrot.pkl', 'rb') as f:
# ...   mynewlist = pickle.load(f)


In [27]:
 df,topics, probs, model = topicModellingBERT(df=final_merged_reviews,columns = {"customer_sentiment":'POSITIVE','topic':"1"} ,textCol="cleaned_data",path=MODEL_DIR,model_name='positive_topic_modelling')


{'customer_sentiment': 'POSITIVE'}
dataet length:  (21275, 20) 



Batches:   0%|          | 0/665 [00:00<?, ?it/s]

2023-04-30 13:45:07,104 - BERTopic - Transformed documents to Embeddings
2023-04-30 13:45:25,638 - BERTopic - Reduced dimensionality
2023-04-30 13:45:34,154 - BERTopic - Clustered reduced embeddings


    Topic  Count                                               Name
0      -1   9269          -1_excellent_service_convenient_brilliant
1       0   3452                   0_account_money_payment_transfer
2       1    564   1_excellent excellent_excellent_positive_overall
3       2    452                2_great great_great_yeah_like great
4       3    416        3_quick quick_quick_quick issue_issue quick
5       4    363                         4_cheque_photo_picture_bad
6       5    347               5_great great_great_trouble_actually
7       6    304          6_intuitive_seamless_fast service_service
8       7    290         7_perfect_superb_amazing_brilliant perfect
9       8    282          8_experience_service service_service_love
10      9    278                   9_super_love_fine_efficient safe
11     10    265                         10_tin_save slow_save_love
12     11    257         11_service_customer_staff_customer service
13     12    241            12_device_transfer_p

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [19]:
model.visualize_topics()

In [21]:
model.reduce_topics(df['cleaned_data'],nr_topics=8)

2023-04-30 13:29:48,851 - BERTopic - Reduced number of topics from 42 to 8


In [30]:
model.visualize_barchart()